In [1]:
%cd /home/druhe/rail1/src/cnf
%load_ext autoreload
%autoreload 2

import os
os.environ['DATAROOT'] = '/home/druhe/datasets'


/home/druhe/rail1/src/cnf


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
class PermutationBilinear(nn.Module):

    def __init__(self, input_dim, output_dim, num_heads):
        super().__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_heads = num_heads

        # self.weight = nn.Parameter(torch.randn(num_heads * output_dim, input_dim) / input_dim ** 0.5)
        # self.bias = nn.Parameter(torch.zeros(num_heads * output_dim))

        self.linear = nn.Linear(input_dim, output_dim * num_heads)


    def forward(self, x):
        B, H, C = x.size()
        x = x.transpose(2, 1)
        x = self.linear(x)

        x = x.reshape(B, H, self.num_heads, self.output_dim)
        x = x.transpose(1, 2)

        





        


data = torch.randn(32, 1024, 3)
perm = torch.randperm(1024)
data_perm = data[:, perm, :]

ip = torch.einsum('bik, bjk -> bij', data, data)
ip_perm = torch.einsum('bik, bjk -> bij', data_perm, data_perm)


bil = PermutationBilinear(1024, 512, 2)


In [41]:
z = bil(data)
z_perm = bil(data_perm)

torch.Size([32, 3, 1024])
torch.Size([32, 3, 1024])


In [39]:

z[:, perm] - z_perm

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.1458e-06,  4.5300e-06, -8.2254e-06,  ...,  2.5034e-06,
          4.7684e-07,  6.4820e-06],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00, -7.2957e-07, -7.1526e-07,  ...,  4.7684e-07,
          3.5763e-07, -4.7684e-07],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.3842e-07, -2.9802e-08, -4.7684e-07,  ..., -9.5367e-07,
         -1.1921e-07,  5.9605e-08]])

In [72]:
# [B, L, 3] -> [B, L / 2, 3]


# [B, L, 1, 3] -> # [B, L, L/2, 3]

# [B, L, L / 2, 3] x [B, L, 3] -> [B, L / 2, C]


class PermutationBilinear(nn.Module):

    def __init__(self, input_dim, output_dim, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_heads = num_heads
        # self.bilinear = nn.Bilinear(input_dim, input_dim, output_dim)
        self.weight = nn.Parameter(torch.randn(output_dim, input_dim) / input_dim ** 0.5)

        # W [L / 2, L] steerable
        # Can do attention [L, C] -> [L, C / 2]]

    def forward(self, input):
        B, L, C = input.size()
        h = input.unsqueeze(2).expand(B, L, self.num_heads, C) # Can be done with linear [C -> C * num_heads]
        energy = torch.einsum('blhc, oc, blc -> bloh', h, self.weight, input)

        attn = F.softmax(energy, dim=1)

        output = torch.einsum('bloh, blhc -> bho', attn, h)

        return output


        



pmb = PermutationBilinear(3, 3, 2)

output = pmb(data)
output_perm = pmb(data_perm)

output - output_perm

        

        






                




tensor([[[-5.3644e-07, -4.7684e-07, -7.1526e-07],
         [-5.3644e-07, -4.7684e-07, -7.1526e-07]],

        [[ 2.1458e-06, -6.1989e-06, -8.3447e-07],
         [ 2.1458e-06, -6.1989e-06, -8.3447e-07]],

        [[ 1.3113e-06,  3.8147e-06,  2.3842e-06],
         [ 1.3113e-06,  3.8147e-06,  2.3842e-06]],

        [[-8.3447e-07, -4.7684e-07, -2.3842e-07],
         [-8.3447e-07, -4.7684e-07, -2.3842e-07]],

        [[ 5.9605e-08, -5.3644e-07, -1.7881e-06],
         [ 5.9605e-08, -5.3644e-07, -1.7881e-06]],

        [[-1.4305e-06, -2.3842e-06, -4.7684e-06],
         [-1.4305e-06, -2.3842e-06, -4.7684e-06]],

        [[ 3.8147e-06, -1.0729e-06,  2.3842e-07],
         [ 3.8147e-06, -1.0729e-06,  2.3842e-07]],

        [[ 1.1921e-06, -1.9073e-06,  2.9802e-08],
         [ 1.1921e-06, -1.9073e-06,  2.9802e-08]],

        [[ 2.3842e-06, -3.5763e-06, -2.3842e-07],
         [ 2.3842e-06, -3.5763e-06, -2.3842e-07]],

        [[ 1.1921e-07,  4.4703e-07,  4.7684e-07],
         [ 1.1921e-07,  4.4703e-

In [62]:
attn = MultiHeadAttention(3, 1)

In [63]:
attn(data, data, data, None)

torch.Size([32, 1, 1024, 1024]) torch.Size([32, 1024, 1, 3])


RuntimeError: No active exception to reraise

In [90]:
x = torch.randn(32, 1024, 3)

in_channels = 4
out_channels = 8
num_heads = 512 

Q_lin = nn.Linear(in_channels, out_channels * num_heads)
K_lin = nn.Linear(in_channels, out_channels * num_heads)
V_lin = nn.Linear(in_channels, out_channels * num_heads)

Q = Q_lin(x)
K = K_lin(x)
V = V_lin(x)

attn_layer = nn.MultiheadAttention(out_channels * num_heads, 64)

output, weights = attn_layer(Q, K, V)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32768x3 and 4x4096)

In [89]:
output = output.sum(1).reshape(32, num_heads, out_channels)

RuntimeError: shape '[32, 512, 6]' is invalid for input of size 192

In [87]:
output.shape

torch.Size([32, 512, 6])

In [86]:
class MultiHeadAttention(nn.Module):

    def __init__(self, in_channels, out_channels, num_heads):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_heads = num_heads

        self.Q_lin = nn.Linear(in_channels, out_channels * num_heads)
        self.K_lin = nn.Linear(in_channels, out_channels * num_heads)
        self.V_lin = nn.Linear(in_channels, out_channels * num_heads)

        self.attn_layer = nn.MultiheadAttention(out_channels * num_heads, 64)

    def forward(self, x):
        Q = self.Q_lin(x)
        K = self.K_lin(x)
        V = self.V_lin(x)

        output, weights = self.attn_layer(Q, K, V)

        output = output.sum(1).reshape(32, self.num_heads, self.out_channels)

        return output

In [4]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.layer1 = MultiHeadAttention(4, 8, 512)
        self.layer2 = MultiHeadAttention(8, 16, 256)
        self.layer3 = MultiHeadAttention(16, 32, 128)
        self.layer4 = MultiHeadAttention(32, 64, 64)
        self.layer5 = MultiHeadAttention(64, 128, 32)
        self.layer6 = MultiHeadAttention(128, 256, 16)
        self.layer7 = MultiHeadAttention(256, 512, 8)
        self.layer8 = MultiHeadAttention(512, 1024, 4)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)

        return x

In [95]:
network = Network().cuda()
input = torch.randn(32, 1024, 4).cuda()
output = network(input)

In [94]:
output.shape

torch.Size([32, 4, 1024])

In [5]:
class PermutationBilinear(nn.Module):

    def __init__(self, input_dim, output_dim, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_heads = num_heads
        self.weight = nn.Parameter(torch.randn(output_dim, input_dim) / input_dim ** 0.5)

        # W [L / 2, L] steerable
        # Can do attention [L, C] -> [L, C / 2]]

    def forward(self, input):
        B, L, C = input.size()
        h = input.unsqueeze(2).expand(B, L, self.num_heads, C) # Can be done with linear [C -> C * num_heads]
        energy = torch.einsum('blhc, oc, blc -> bloh', h, self.weight, input)

        attn = F.softmax(energy, dim=1)

        output = torch.einsum('bloh, blhc -> bho', attn, h)

        return output

In [10]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.layer1 = PermutationBilinear(4, 8, 512)
        self.layer2 = PermutationBilinear(8, 16, 256)
        self.layer3 = PermutationBilinear(16, 32, 128)
        self.layer4 = PermutationBilinear(32, 64, 64)
        self.layer5 = PermutationBilinear(64, 128, 32)
        self.layer6 = PermutationBilinear(128, 256, 16)
        self.layer7 = PermutationBilinear(256, 512, 8)
        self.layer8 = PermutationBilinear(512, 1024, 4)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)

        return x 
    
network = Network()
input = torch.randn(32, 1024, 4)
output = network(input)

In [ ]:
output.shape

torch.Size([32, 4, 1024])

In [13]:
input = torch.randn(32, 1024, 2, 3)

In [14]:
# [B, L, K, C] # knn

# [B, L, C'] # message passing

# [B, L, L'] # Get permutation- steerable kernel

# [B, L / 2, C'] # Apply permutation- steerable kernel







In [ ]:
# [B, L, L, 1] cdist
# [B, L, L, C] apply kernel
# [B, L, C] sum
# [B, L, L'] get permutation-steerable kernel
# [B, L, C] apply kernel



In [40]:
class Layer(nn.Module):
    def __init__(self, channels, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.attn = nn.MultiheadAttention(channels, num_heads, batch_first=True, vdim=2*channels, kdim=2 * channels,)



    def forward(self, x):

        return self.attn(x, x, x)[0]

x = torch.randn(32, 1024, 3)
layer = Layer(3, 1)
output = layer(x)

AssertionError: expecting key weights shape of (3, 3), but got torch.Size([3, 6])

In [45]:
x = torch.randn(32, 1024, 3)


w = torch.randn(512, 3)

x_w = F.linear(x, w)

In [50]:
x_s = torch.einsum('bic, bik -> bkc', x, x_w)

In [55]:
attn = nn.MultiheadAttention(3, 1, batch_first=True)

res, _ = attn(x_s, x, x)

In [308]:
d = 4
a = torch.randn(d, 3)
perm = torch.randperm(d)

a_perm = a[perm]
# cdist = torch.cdist(a, a)
# cdist_perm = torch.cdist(a_perm, a_perm)

a_norm = torch.norm(a, dim=1)
a_perm_norm = torch.norm(a_perm, dim=1)

w = torch.randn(1, d)
# w = torch.ones(1, d)

In [313]:
d = torch.linspace(-1, 1, 4)


d


In [298]:
a_norm = a_norm[:, None] ** 2 @ wd
a_perm_norm = a_perm_norm[:, None] ** 2 @ w

In [299]:
a_norm, a_perm_norm

(tensor([[ 1.2649, -0.4269,  0.2583,  0.3775],
         [ 2.2522, -0.7602,  0.4598,  0.6721],
         [ 2.8475, -0.9611,  0.5814,  0.8498],
         [ 3.0734, -1.0374,  0.6275,  0.9172]]),
 tensor([[ 1.2649, -0.4269,  0.2583,  0.3775],
         [ 2.2522, -0.7602,  0.4598,  0.6721],
         [ 2.8475, -0.9611,  0.5814,  0.8498],
         [ 3.0734, -1.0374,  0.6275,  0.9172]]))

In [300]:
proc = a_norm * a_norm
proc_perm = a_perm_norm * a_perm_norm

In [301]:
proc, proc_perm

(tensor([[1.5999, 0.1823, 0.0667, 0.1425],
         [5.0724, 0.5779, 0.2115, 0.4518],
         [8.1083, 0.9238, 0.3380, 0.7221],
         [9.4458, 1.0761, 0.3938, 0.8413]]),
 tensor([[1.5999, 0.1823, 0.0667, 0.1425],
         [5.0724, 0.5779, 0.2115, 0.4518],
         [8.1083, 0.9238, 0.3380, 0.7221],
         [9.4458, 1.0761, 0.3938, 0.8413]]))

In [302]:
is_max = proc == proc.max(1).values[:, None]
is_max_perm = proc_perm == proc_perm.max(1).values[:, None]

In [303]:
is_max

tensor([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False]])

In [304]:
is_max_perm

tensor([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False]])

In [138]:
res.shape

torch.Size([4, 4])

In [135]:
mask

tensor([2.0210, 2.0210, 2.0210, 0.0000])

In [366]:
x = torch.tensor([
[-0.1, 1],
[0.1, 1],
[-0.1, -1],
[0.1, -1],
]
)

torch.cdist(x, x)

tensor([[0.0000, 0.2000, 2.0000, 2.0100],
        [0.2000, 0.0000, 2.0100, 2.0000],
        [2.0000, 2.0100, 0.0000, 0.2000],
        [2.0100, 2.0000, 0.2000, 0.0000]])

In [367]:
# Step 1
inner = torch.einsum('ni, mi->nm', x, x)

inner - torch.diag(x.norm(dim=-1))


tensor([[ 0.0050,  0.9900, -0.9900, -1.0100],
        [ 0.9900,  0.0050, -1.0100, -0.9900],
        [-0.9900, -1.0100,  0.0050,  0.9900],
        [-1.0100, -0.9900,  0.9900,  0.0050]])

In [368]:
inner

tensor([[ 1.0100,  0.9900, -0.9900, -1.0100],
        [ 0.9900,  1.0100, -1.0100, -0.9900],
        [-0.9900, -1.0100,  1.0100,  0.9900],
        [-1.0100, -0.9900,  0.9900,  1.0100]])

In [372]:
inner.softmax(dim=-1)

tensor([[0.4448, 0.4360, 0.0602, 0.0590],
        [0.4360, 0.4448, 0.0590, 0.0602],
        [0.0602, 0.0590, 0.4448, 0.4360],
        [0.0590, 0.0602, 0.4360, 0.4448]])

In [371]:
torch.diag(x.norm(dim=-1) ** 2)

tensor([[1.0100, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0100, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0100, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0100]])

In [378]:
perm = torch.randperm(8)

x = torch.randn(8, 3)
x_perm = x[perm]
w = torch.randn(3, 4)

xw = x @ w
xw_perm = x_perm @ w

In [379]:
y = xw.T @ x
y_perm = xw_perm.T @ x_perm

In [388]:
ip = torch.einsum('ni, mi->nm', x, y)
ip_perm = torch.einsum('ni, mi->nm', x_perm, y_perm)

In [387]:
ip.max(0)

torch.return_types.max(
values=tensor([16.2656, 14.4808, 20.3985,  8.9729]),
indices=tensor([7, 0, 6, 6]))

In [390]:
ip_perm.max(0)

torch.return_types.max(
values=tensor([16.2656, 14.4808, 20.3985,  8.9729]),
indices=tensor([1, 5, 6, 6]))

In [391]:
ip

tensor([[  0.3709,  14.4808,   9.5594,   2.5053],
        [  4.2277,   0.4717,  -1.7566,  -0.9818],
        [ 14.0354,  10.2816,  -4.7372,  -3.5390],
        [ -9.3594,  -7.3979,   2.2389,   2.0479],
        [ -4.2079,  -3.3659,   3.9725,   2.0707],
        [-10.8712,  -0.3860,   9.8752,   4.5297],
        [-19.6837,   1.8114,  20.3985,   8.9729],
        [ 16.2656,  -4.2352, -15.2898,  -6.5882]])

In [392]:
ip_perm

tensor([[-10.8712,  -0.3860,   9.8752,   4.5297],
        [ 16.2656,  -4.2353, -15.2898,  -6.5882],
        [ 14.0354,  10.2816,  -4.7372,  -3.5390],
        [  4.2277,   0.4717,  -1.7566,  -0.9818],
        [ -4.2079,  -3.3659,   3.9725,   2.0707],
        [  0.3709,  14.4808,   9.5594,   2.5053],
        [-19.6837,   1.8114,  20.3985,   8.9729],
        [ -9.3594,  -7.3979,   2.2389,   2.0479]])

In [403]:
torch.cdist(x, x).sum(-1).topk(3, largest=False)

torch.return_types.topk(
values=tensor([10.4142, 11.1507, 11.1549]),
indices=tensor([1, 5, 4]))

In [408]:
import torch

def kmeans(X, num_clusters, num_iterations, device='cpu', tol=1e-4):
    # Initialize centroids randomly from the data points
    indices = torch.randperm(X.shape[0])[:num_clusters]
    centroids = X[indices].to(device)
    prev_error = None

    for _ in range(num_iterations):
        # Assign each data point to the closest centroid
        distances = torch.cdist(X.to(device), centroids)
        closest = distances.argmin(dim=1)
        error = 0

        # Update centroids and compute the error
        print(closest)
        raise
        for i in range(num_clusters):
            cluster_points = X[closest == i]
            if len(cluster_points) > 0:
                new_centroid = cluster_points.mean(dim=0)
                error += torch.sum((cluster_points - new_centroid).pow(2))
                centroids[i] = new_centroid

        # Check for convergence
        if prev_error is not None and torch.abs(prev_error - error) < tol:
            print(f"Converged at iteration {_+1}")
            break

        prev_error = error

    return centroids, closest

# Example usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = torch.randn(1000, 3)  # 1000 data points with 2 features each
num_clusters = 5
num_iterations = 100

centroids, assignments = kmeans(X, num_clusters, num_iterations, device)


tensor([1, 3, 4, 2, 4, 1, 3, 3, 1, 4, 4, 0, 1, 4, 0, 4, 1, 3, 4, 4, 4, 0, 3, 3,
        2, 3, 2, 4, 1, 4, 0, 1, 3, 3, 3, 1, 0, 1, 0, 0, 3, 4, 0, 1, 3, 3, 0, 3,
        1, 4, 4, 0, 1, 1, 1, 3, 4, 2, 4, 4, 0, 1, 2, 3, 4, 1, 1, 4, 0, 1, 1, 2,
        0, 2, 4, 0, 1, 2, 1, 1, 0, 4, 3, 4, 0, 0, 1, 4, 1, 4, 2, 1, 0, 1, 4, 1,
        0, 0, 0, 0, 1, 2, 4, 3, 3, 0, 1, 1, 0, 0, 4, 4, 4, 1, 4, 3, 0, 1, 4, 3,
        1, 1, 1, 4, 1, 2, 4, 1, 3, 0, 4, 1, 4, 1, 0, 4, 4, 1, 2, 4, 4, 4, 4, 0,
        2, 1, 3, 1, 0, 0, 1, 4, 3, 4, 0, 1, 4, 4, 0, 0, 2, 1, 4, 0, 4, 2, 4, 0,
        1, 1, 4, 0, 0, 3, 4, 1, 1, 3, 2, 4, 1, 1, 1, 1, 1, 1, 0, 3, 1, 0, 2, 2,
        4, 0, 4, 3, 4, 0, 0, 2, 4, 2, 1, 0, 0, 4, 4, 4, 2, 1, 3, 4, 1, 1, 1, 3,
        0, 1, 1, 0, 1, 4, 0, 0, 3, 4, 2, 4, 4, 4, 4, 1, 4, 0, 1, 0, 3, 1, 1, 2,
        1, 0, 4, 4, 4, 2, 1, 4, 4, 1, 0, 0, 4, 0, 3, 4, 1, 2, 4, 1, 0, 0, 0, 4,
        0, 1, 1, 3, 4, 4, 4, 4, 1, 3, 4, 4, 2, 2, 0, 0, 0, 1, 4, 4, 4, 0, 1, 1,
        4, 0, 4, 3, 2, 1, 1, 1, 4, 0, 4,

RuntimeError: No active exception to reraise

In [409]:
x = torch.randn(32, 3)

In [410]:
n = x.norm(dim=-1)

In [412]:
ma, mi = n.argmax(0), n.argmin(0)

In [415]:
x[ma], x[mi], x.mean(dim=0)

(tensor([-1.1554,  3.1600,  0.4453]),
 tensor([ 0.0844, -0.2514,  0.2357]),
 tensor([-0.2523, -0.1596, -0.1337]))

: 